In [1]:
from function import pdb2pqr
input_path = './data/pdbs'
output_path = './data/pqrs'
pdb2pqr(input_path = input_path, output_path = output_path)

" from function import pdb2pqr\ninput_path = './data/pdbs'\noutput_path = './data/pqrs'\npdb2pqr(input_path = input_path, output_path = output_path) "

In [ ]:
import numpy as np
import trimesh
protein_coords = np.array###!!!!!!!!!!!!!

mesh = trimesh.Trimesh(protein_coords, np.ones((protein_coords.shape[0], 3)))
alpha = 0.4
mesh.alpha_shape(alpha)
surface_area = mesh.area
print("Estimated surface area of the protein:", surface_area)

In [1]:
import os
import numpy as np

path = './data/pqrs'
pqr_file = 'C0H3V8.pqr'
Atom_array = np.empty((0, 5))
with open(os.path.join(path, str(pqr_file))) as f:
    for line in f:
        line = line.replace('-', '  -')
        if line.startswith('ATOM'):
            line_array = np.array([[int(line.split()[1]), float(line.split()[5]), float(line.split()[6]), float(line.split()[7]), float(line.split()[9])]]) 

            Atom_array = np.append(Atom_array, line_array, axis = 0)
print(Atom_array.shape)

(1117, 5)


In [41]:
from helper_function import distance

def calculate_protein_volume(Atom_array, cutoff):
  """
  Calculates the approximate volume of a protein using the union boundary approach.

  Args:
      Atom_array: A NumPy array with atom information (columns: number, x, y, z, radius).
      cutoff: The distance threshold defining the protein boundary (Å).

  Returns:
      The approximate volume of the protein (Å^3).
  """

  # Calculate distances (lower triangular part only)
  dist = distance(Atom_array, Atom_array, cutoff = cutoff, remove_nan=False)
  dist = np.tril(dist)  # Keep only lower triangular part

  # Calculate individual atom volumes
  vol_A = 4/3 * np.pi * Atom_array[:, 4]**3  # Volume of each atom (assuming spherical)

  # Calculate overlapping volume using vectorization and broadcasting
  dist = np.nan_to_num(dist)  # Replace NaNs with zeros
  d = dist[1:,1:]  # Select non-zero distances
  r1 = Atom_array[:, 4]  # Reshape radii for broadcasting
  r2 = Atom_array[:, 4]  # Select radii for overlapping atoms
  vol_overlap_array = (np.pi * (r2 + r1 - d)**2 * (d**2 + 2*d*r1 - 3*r1**2 + 2*d*r2 + 6*r1*r2 - 3*r2**2)) / (12*d)
  vol_overlap_array = np.nan_to_num(vol_overlap_array)  # Replace NaNs with zeros
  vol_overlap = np.sum(vol_overlap_array)

  # Total protein volume (overlap minus individual volumes)
  Prot_vol = vol_overlap - np.sum(vol_A)
  return Prot_vol

# Example usage
cutoff = np.max(Atom_array[:, 4])*2  # Adjust cutoff as needed
Prot_vol = calculate_protein_volume(Atom_array, cutoff)
print("Approximate protein volume:", Prot_vol, "Å^3")


Approximate protein volume: inf Å^3


C:\Users\marik\AppData\Local\Temp\ipykernel_25132\4285782273.py:27: RuntimeWarning: divide by zero encountered in divide
  vol_overlap_array = (np.pi * (r2 + r1 - d)**2 * (d**2 + 2*d*r1 - 3*r1**2 + 2*d*r2 + 6*r1*r2 - 3*r2**2)) / (12*d)
C:\Users\marik\AppData\Local\Temp\ipykernel_25132\4285782273.py:27: RuntimeWarning: invalid value encountered in divide
  vol_overlap_array = (np.pi * (r2 + r1 - d)**2 * (d**2 + 2*d*r1 - 3*r1**2 + 2*d*r2 + 6*r1*r2 - 3*r2**2)) / (12*d)


In [43]:
from helper_function import distance
from scipy.sparse import lil_matrix

dist = distance(Atom_array, Atom_array, cutoff = np.max(Atom_array[:, 4])*2, remove_nan = False)
dist2 = dist[1:,1:]
dist2[np.triu_indices(dist2.shape[0], k = 1)] = np.nan
dist2 = np.nan_to_num(dist2)
dist3 = lil_matrix(dist2)
print(dist3.shape)
print(dist.shape)
print(Atom_array.shape)

vol = dist
vol[1:,1:] = np.nan
for n in range(Atom_array.shape[0]):
    for m in range(Atom_array.shape[0]):
        if dist3[n,m] != 0:
            d = dist3[n,m]
            r1 = Atom_array[n, 4]
            r2 = Atom_array[m, 4]
            vol[n+1,m+1] = (np.pi*(r2+r1-d)**2 * (d**2 + 2*d*r1 - 3*r1**2 + 2*d*r2 + 6*r1*r2 - 3*r2**2))/(12*d)
vol_A = 0
for n in range(Atom_array.shape[0]):
        vol_A += 4/3 * np.pi * Atom_array[n, 4]**3

vol = np.nan_to_num(vol)
vol_overlap = np.sum(vol)

Prot_vol = vol_A - vol_overlap
print(Prot_vol)

(1117, 1117)
(1118, 1118)
(1117, 5)
-1273736.3217520753


In [ ]:
import numpy as np

# Assume arr is your symmetric numpy array
arr = np.array([[1, 2, 3], [2, 4, 5], [3, 5, 6]])

# Get the upper triangle indices
i_upper = np.triu_indices(arr.shape[0], 1)

# Extract the upper triangle values
unique_values = arr[i_upper]

print(unique_values)

Sphere function

In [13]:
def sphere(Atom_array, phi_resolution=100, theta_resolution=100):
  """
  Generates sphere points for each atom in Atom_array and appends them as new columns.

  Args:
      Atom_array: A NumPy array with atom information (x, y, z, radius in columns).
      phi_resolution: Number of points discretizing the colatitude angle (0 to pi).
      theta_resolution: Number of points discretizing the longitude angle (0 to 2pi).

  Returns:
      A modified NumPy array with additional columns for sphere points.
  """

  # Extract center coordinates and radius from each atom row
  a = Atom_array[:, 1]
  b = Atom_array[:, 2]
  c = Atom_array[:, 3]
  r = Atom_array[:, 4]

  # Generate sphere points for each atom
  phi = np.linspace(0, np.pi, phi_resolution)
  theta = np.linspace(0, 2 * np.pi, theta_resolution)

  # Calculate x, y, and z coordinates for each atom's sphere
  x = a[:, np.newaxis] + r * np.cos(theta) * np.sin(phi)  # Add newaxis for broadcasting
  y = b[:, np.newaxis] + r * np.sin(theta) * np.sin(phi)
  z = c[:, np.newaxis] + r * np.cos(phi)

  # Reshape x, y, and z to match the number of atoms (rows in Atom_array)
  x = x.reshape(-1, x.shape[-1])  # Reshape to remove unnecessary dimension
  y = y.reshape(-1, y.shape[-1])
  z = z.reshape(-1, z.shape[-1])

  # Concatenate Atom_array with the new sphere point columns
  new_array = np.concatenate((Atom_array, x, y, z), axis=1)

  return new_array

atom volume function

In [32]:
def atom_volume(Atom_array):
    """
    Calculate the volume of each atom in the Atom_array.
    
    Args:
        Atom_array: A NumPy array with atom information (x, y, z, radius in columns).
        
    Returns:
        A NumPy array with the volume of each atom appended as a new column.
    """
    
    # Calculate the volume of each atom
    volume = 4/3 * np.pi * Atom_array[:, 4]**3
    
    # Append the volume as a new column
    Atom_array = np.concatenate((Atom_array, volume[:, np.newaxis]), axis=1)
    
    return Atom_array

intersect_vol update

In [69]:
def intersect_vol(array, r1:str, r2:str, dis = None):   
    import numpy as np
    r1 = float(r1)
    r2 = float(r2)
    vol = array
    if dis is not None:
        d = dis
    else:
        d = array[1:,1:]
    vol[1:,1:] = (np.pi*(r2+r1-d)**2 * (d**2 + 2*d*r1 - 3*r1**2 + 2*d*r2 + 6*r1*r2 - 3*r2**2))/(12*d)
    return vol

In [80]:

def overlap_with_kdTree(Atom_array):
    """
    Estimates the overlapping volume using KDTree for efficient neighbor search.

    Args:
    Atom_array: A NumPy array with sphere data (center coordinates and radius in rows)

    Returns:
    The estimated total overlapping volume.
    """
    from scipy.spatial import KDTree
    
    centers = Atom_array[:, 1:3]
    # Build KDTree from sphere centers
    tree = KDTree(centers)
    total_overlap = 0
    for i, center in enumerate(centers):
        # Find neighbor spheres within maximum reach (considering radii)
        neighbors = tree.query_ball_point(center, r=np.max(Atom_array[:, 3]))[1]
        # Exclude the query sphere itself from neighbors
        #neighbors = neighbors[neighbors != i]
    return neighbors


"""     for i in range(neighbors):
        # Calculate overlap volume with neighbor (similar to pairwise_overlap function)
        r1 = Atom_array[i,4]
        vol = (np.pi*(r2+r1-d)**2 * (d**2 + 2*d*r1 - 3*r1**2 + 2*d*r2 + 6*r1*r2 - 3*r2**2))/(12*d)

        neighbor_center = Atom_array[neighbor, :3]
        neighbor_radius = Atom_array[neighbor, 4]
        total_overlap += intersect_vol(, neighbor_radius,)  # Call a separate function """



'     for i in range(neighbors):\n        # Calculate overlap volume with neighbor (similar to pairwise_overlap function)\n        r1 = Atom_array[i,4]\n        vol = (np.pi*(r2+r1-d)**2 * (d**2 + 2*d*r1 - 3*r1**2 + 2*d*r2 + 6*r1*r2 - 3*r2**2))/(12*d)\n\n        neighbor_center = Atom_array[neighbor, :3]\n        neighbor_radius = Atom_array[neighbor, 4]\n        total_overlap += intersect_vol(, neighbor_radius,)  # Call a separate function '

In [83]:
test = overlap_with_kdTree(Atom_array)
print(test)

1104


In [ ]:
def vol_calc_it()

In [78]:
n_test = overlap_with_kdTree(Atom_array)


ValueError: operands could not be broadcast together with shapes (2,) (3,) 

In [46]:
radius=np.max(Atom_array[:, 4])*2
print(radius)

4.0


In [38]:
max_value = np.max(Atom_array[:, 4])
print("Maximum radius of an atom:", max_value)

Maximum radius of an atom: 2.0


In [37]:
from helper_function import distance
test_vol = atom_volume(Atom_array)
test_dist = distance(test_vol, test_vol, remove_nan = False)

for n in range(len(test_dist)):
    for i in range(len(test_dist[n])):


(1117, 6)
(1118, 1118)
